# BLM calculator

Steps this pipeline requieres:
1. Run the same scenario 5 times, 10 runs each, modifying the BLM value according to: 
BLM = Z*sqrt(PU area)
(Z = 0.01, 0.1,1, 10 and 100)
2. Plot the best solution of each of these scenario outputs
3. Plot BLM vs Cost
4. Calculate the optimum BLM value that minimizes both cost and BLM (clumping)

## 1. Run the same scenario 5 times

In [9]:
# insert code here
MARXAN_FOLDER = '/home/jovyan/work/datasets/raw/Marxan_BLM'
MARXAN_EXECUTABLE = f'{MARXAN_FOLDER}/MarOpt_v243_Linux64'
MARXAN_INPUTDATA = 'input.dat'

In [17]:
#!mkdir $MARXAN_FOLDER/output_3
!ls $MARXAN_FOLDER

input		      Marxan.exe      output_0.1  output_100	script.R
input.dat	      Marxan_x64.exe  output_1	  project1.zcp	scriptR.bat
MarOptTotalAreas.csv  output_0.01     output_10   pu


In [16]:
!rm -r $MARXAN_FOLDER/output_3
!rm -r $MARXAN_FOLDER/output_2

## Plot best solution of each

and some more text

## Plot BLM vs Cost

and some more text

## Calculate optimum BLM

and some more text